In [1]:
import pandas as pd

df_canada = pd.read_csv('https://raw.githubusercontent.com/muzabu51/Coursera_Capstone/master/Canada_post_lat_long.csv')
df_canada = df_canada[df_canada['Borough'].str.contains('Toronto')]
df_canada

,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [2]:
import geocoder
from geopy.geocoders import Nominatim 

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<h5>Map of Toronto and plotting 'Toronto' containing boroughs

In [3]:
import folium
# create map of Canada using latitude and longitude values
canada_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_canada['Latitude'], df_canada['Longitude'], df_canada['Borough'], df_canada['Neighborhood']):
    label = f'{neighborhood}, {borough}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc',
        fill_opacity=0.7, parse_html=False).add_to(canada_map)  
canada_map

<h6>Defining FourSquare API credentials

In [4]:
CLIENT_ID = 'DKCJQ0FDTYGLBC32I1EFEEH2OKFB01VCALJRHZLAVDMFCKWB' # your Foursquare ID
CLIENT_SECRET = 'AMFT3Q2Q53AXH2BNVI0NHJL4XJKXK3SGDRAVMPMA3LGGZ0PV' # your Foursquare Secret
VERSION = '20200705' # Foursquare API version
LIMIT = 100

<h5>Exploring neighborhoods in Canada

In [14]:
import requests

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}\
&v={VERSION}&ll={latitude},{longitude}&radius={radius}&limit={LIMIT}'
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
canada_venues = getNearbyVenues(names=df_canada['Neighborhood'], latitudes=df_canada['Latitude'], longitudes=df_canada['Longitude'])
print(canada_venues.shape)
canada_venues.head()

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,The Beaches,43.676357,-79.293031,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,The Beaches,43.676357,-79.293031,Chatime 日出茶太,43.655542,-79.384684,Bubble Tea Shop
3,The Beaches,43.676357,-79.293031,Textile Museum of Canada,43.654396,-79.386500,Art Museum
4,The Beaches,43.676357,-79.293031,Indigo,43.653515,-79.380696,Bookstore


In [16]:
print(f'There are {len(canada_venues["Venue Category"].unique())} unique categories.')
canada_venues.groupby('Neighborhood').count()

There are 57 unique categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,77,77,77,77,77,77
"Brockton, Parkdale Village, Exhibition Place",77,77,77,77,77,77
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",77,77,77,77,77,77
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",77,77,77,77,77,77
Central Bay Street,77,77,77,77,77,77
Christie,77,77,77,77,77,77
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",77,77,77,77,77,77
Davisville,77,77,77,77,77,77


<h5>Analyzing each nieghborhood

In [17]:
# one hot encoding
canada_onehot = pd.get_dummies(canada_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
canada_onehot['Neighborhood'] = canada_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [canada_onehot.columns[-1]] + list(canada_onehot.columns[:-1])
canada_onehot = canada_onehot[fixed_columns]

canada_onehot.head()

,Women's Store,American Restaurant,Art Museum,Bank,Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,Clothing Store,...,Smoothie Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
canada_grouped = canada_onehot.groupby('Neighborhood').mean().reset_index()
canada_grouped.head()

,Neighborhood,Women's Store,American Restaurant,Art Museum,Bank,Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,...,Smoothie Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,Berczy Park,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,...,0.012987,0.025974,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.012987,0.012987
1,"Brockton, Parkdale Village, Exhibition Place",0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,...,0.012987,0.025974,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.012987,0.012987
2,"Business reply mail Processing Centre, South C...",0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,...,0.012987,0.025974,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.012987,0.012987
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,...,0.012987,0.025974,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.012987,0.012987
4,Central Bay Street,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,...,0.012987,0.025974,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.012987,0.012987


In [19]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
canada_grouped = canada_onehot.groupby('Neighborhood').mean().reset_index()
canada_grouped.head()

,Neighborhood,Women's Store,American Restaurant,Art Museum,Bank,Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,...,Smoothie Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,Berczy Park,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,...,0.012987,0.025974,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.012987,0.012987
1,"Brockton, Parkdale Village, Exhibition Place",0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,...,0.012987,0.025974,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.012987,0.012987
2,"Business reply mail Processing Centre, South C...",0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,...,0.012987,0.025974,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.012987,0.012987
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,...,0.012987,0.025974,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.012987,0.012987
4,Central Bay Street,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,...,0.012987,0.025974,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.012987,0.012987


In [27]:
num_top_venues = 5

for hood in canada_grouped['Neighborhood']:
    temp = canada_grouped[canada_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))

             venue  freq
0   Clothing Store  0.08
1      Coffee Shop  0.08
2       Restaurant  0.04
3          Theater  0.03
4  Thai Restaurant  0.03
             venue  freq
0   Clothing Store  0.08
1      Coffee Shop  0.08
2       Restaurant  0.04
3          Theater  0.03
4  Thai Restaurant  0.03
             venue  freq
0   Clothing Store  0.08
1      Coffee Shop  0.08
2       Restaurant  0.04
3          Theater  0.03
4  Thai Restaurant  0.03
             venue  freq
0   Clothing Store  0.08
1      Coffee Shop  0.08
2       Restaurant  0.04
3          Theater  0.03
4  Thai Restaurant  0.03
             venue  freq
0   Clothing Store  0.08
1      Coffee Shop  0.08
2       Restaurant  0.04
3          Theater  0.03
4  Thai Restaurant  0.03
             venue  freq
0   Clothing Store  0.08
1      Coffee Shop  0.08
2       Restaurant  0.04
3          Theater  0.03
4  Thai Restaurant  0.03
             venue  freq
0   Clothing Store  0.08
1      Coffee Shop  0.08
2       Restaurant  0.04


In [28]:
# getting top 5 for each nighborhood
def returnMostCommonVenues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
import numpy as np
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = canada_grouped['Neighborhood']

for ind in np.arange(canada_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(canada_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Steakhouse
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Steakhouse
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Steakhouse
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Steakhouse
4,Central Bay Street,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Steakhouse


<h5>Clustering neighborhoods

In [34]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

canada_grouped_clustering = canada_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(canada_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/Users/habeeb/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

canada_merged = df_canada

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
canada_merged = canada_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

canada_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Steakhouse
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Steakhouse
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Steakhouse
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Steakhouse
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Coffee Shop,Clothing Store,Restaurant,Japanese Restaurant,Steakhouse


In [36]:
canada_merged[canada_merged['Cluster Labels'].isnull()]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
